In [24]:
import pandas as pd
import joblib
import numpy as np

# 1. Cargar el modelo
print("📦 Cargando modelo...")
pipeline = joblib.load('../models/kmeans_model__oldc.joblib')

# 2. Cargar nuevos datos
print("📊 Cargando datos...")
new_data = pd.read_csv('new_data_sample.csv')

# 3. Función de preprocesamiento (debe coincidir exactamente con el entrenamiento)
def preprocess_for_prediction(df):
    """
    Aplica el mismo preprocesamiento que se usó durante el entrenamiento
    """
    df = df.copy()
    
    # Recodificar variables binarias (2 -> 0)
    binary_cols = ['SEXO', 'TUVO_TRABAJO', 'BUSCA_OTRO_TRABAJO', 'QUIERE_MAS_HORAS', 'SEGURO_SALUD']
    for col in binary_cols:
        if col in df.columns:
            df[col] = df[col].replace({2: 0})
    
    # Crear variable DISCAPACIDAD a partir de las limitaciones
    disability_cols = ['LIMIT_MOVIMIENTO', 'LIMIT_VISION', 'LIMIT_COMUNICACION', 
                      'LIMIT_AUDICION', 'LIMIT_APRENDIZAJE', 'LIMIT_RELACION']
    
    if all(col in df.columns for col in disability_cols):
        df['DISCAPACIDAD'] = df[disability_cols].sum(axis=1).apply(lambda x: 1 if x >= 1 else 0)
        df = df.drop(columns=disability_cols)
    
    # One-hot encoding para variables categóricas
    categorical_cols = ['ETNIA', 'TIPO_TRABAJADOR', 'NIVEL_OCUPACION', 
                       'REGISTRO_SUNAT', 'LIBROS_CONTABLES', 'FRECUENCIA_PAGO']
    
    for col in categorical_cols:
        if col in df.columns:
            df = pd.get_dummies(df, columns=[col], drop_first=True, dummy_na=True)
    
    # Columnas que el modelo espera (basado en el modelo cargado)
    expected_features = [
        'SEXO', 'EDAD', 'NIVEL_EDUCATIVO', 'TUVO_TRABAJO', 'BUSCA_OTRO_TRABAJO',
        'TAMANO_EMPRESA', 'QUIERE_MAS_HORAS', 'INGRESO_PRINCIPAL', 'SEGURO_SALUD',
        'DIAS_AUSENTE', 'DISCAPACIDAD', 'ETNIA_2', 'ETNIA_3', 'ETNIA_4', 'ETNIA_5',
        'ETNIA_6', 'ETNIA_7', 'ETNIA_8', 'ETNIA_9', 'TIPO_TRABAJADOR_2.0',
        'TIPO_TRABAJADOR_3.0', 'TIPO_TRABAJADOR_4.0', 'TIPO_TRABAJADOR_6.0',
        'TIPO_TRABAJADOR_7.0', 'TIPO_TRABAJADOR_8.0', 'TIPO_TRABAJADOR_9.0',
        'NIVEL_OCUPACION_2', 'NIVEL_OCUPACION_3', 'NIVEL_OCUPACION_4',
        'REGISTRO_SUNAT_2.0', 'REGISTRO_SUNAT_3.0', 'LIBROS_CONTABLES_2.0',
        'FRECUENCIA_PAGO_2.0', 'FRECUENCIA_PAGO_3.0', 'FRECUENCIA_PAGO_4.0',
        'FRECUENCIA_PAGO_5.0'
    ]
    
    # Asegurar que todas las columnas esperadas estén presentes
    for col in expected_features:
        if col not in df.columns:
            df[col] = 0
    
    # Retornar solo las columnas en el orden correcto
    return df[expected_features]

# 4. Aplicar preprocesamiento
print("🔧 Aplicando preprocesamiento...")
try:
    processed_data = preprocess_for_prediction(new_data)
    print(f"✅ Datos procesados: {processed_data.shape}")
    print(f"📋 Columnas procesadas: {list(processed_data.columns)}")
    
    # 5. Hacer predicción
    print("🎯 Realizando predicción...")
    clusters = pipeline.predict(processed_data)
    
    # 6. Agregar resultados a los datos originales
    new_data['Cluster'] = clusters
    
    # 7. Guardar resultados
    output_file = 'resultados_con_clusters.csv'
    new_data.to_csv(output_file, index=False)
    
    print("✅ ¡Predicción completada exitosamente!")
    print(f"📈 Distribución de clusters:")
    cluster_counts = pd.Series(clusters).value_counts().sort_index()
    for cluster, count in cluster_counts.items():
        print(f"   Cluster {cluster}: {count} registros")
    
    print(f"💾 Resultados guardados en: {output_file}")
    
    # Mostrar una muestra de los resultados
    print("\n📝 Muestra de resultados:")
    print(new_data[['SEXO', 'EDAD', 'NIVEL_EDUCATIVO', 'Cluster']].head())
    
except Exception as e:
    print(f"🚨 ERROR: {e}")
    print("\n🔍 Información de depuración:")
    print(f"Forma de los datos originales: {new_data.shape}")
    print(f"Columnas disponibles: {list(new_data.columns)}")

📦 Cargando modelo...
📊 Cargando datos...
🔧 Aplicando preprocesamiento...
✅ Datos procesados: (25, 36)
📋 Columnas procesadas: ['SEXO', 'EDAD', 'NIVEL_EDUCATIVO', 'TUVO_TRABAJO', 'BUSCA_OTRO_TRABAJO', 'TAMANO_EMPRESA', 'QUIERE_MAS_HORAS', 'INGRESO_PRINCIPAL', 'SEGURO_SALUD', 'DIAS_AUSENTE', 'DISCAPACIDAD', 'ETNIA_2', 'ETNIA_3', 'ETNIA_4', 'ETNIA_5', 'ETNIA_6', 'ETNIA_7', 'ETNIA_8', 'ETNIA_9', 'TIPO_TRABAJADOR_2.0', 'TIPO_TRABAJADOR_3.0', 'TIPO_TRABAJADOR_4.0', 'TIPO_TRABAJADOR_6.0', 'TIPO_TRABAJADOR_7.0', 'TIPO_TRABAJADOR_8.0', 'TIPO_TRABAJADOR_9.0', 'NIVEL_OCUPACION_2', 'NIVEL_OCUPACION_3', 'NIVEL_OCUPACION_4', 'REGISTRO_SUNAT_2.0', 'REGISTRO_SUNAT_3.0', 'LIBROS_CONTABLES_2.0', 'FRECUENCIA_PAGO_2.0', 'FRECUENCIA_PAGO_3.0', 'FRECUENCIA_PAGO_4.0', 'FRECUENCIA_PAGO_5.0']
🎯 Realizando predicción...
✅ ¡Predicción completada exitosamente!
📈 Distribución de clusters:
   Cluster 0: 10 registros
   Cluster 1: 14 registros
   Cluster 2: 1 registros
💾 Resultados guardados en: resultados_con_cluste

In [11]:
import pandas as pd
import joblib
import numpy as np
from pipeline import 


# 1. Cargar el modelo
print("📦 Cargando modelo...")
pipeline = joblib.load('../models/kmeans_model.joblib')

# 2. Cargar nuevos datos
print("📊 Cargando datos...")
new_data = pd.read_csv('new_data_sample.csv')

# 3. Función de preprocesamiento (debe coincidir exactamente con el entrenamiento)
def preprocess_for_prediction(df):
    """
    Aplica el mismo preprocesamiento que se usó durante el entrenamiento
    """
    df = df.copy()
    
    # Recodificar variables binarias (2 -> 0)
    binary_cols = ['SEXO', 'TUVO_TRABAJO', 'BUSCA_OTRO_TRABAJO', 'QUIERE_MAS_HORAS', 'SEGURO_SALUD']
    for col in binary_cols:
        if col in df.columns:
            df[col] = df[col].replace({2: 0})
    
    # Crear variable DISCAPACIDAD a partir de las limitaciones
    disability_cols = ['LIMIT_MOVIMIENTO', 'LIMIT_VISION', 'LIMIT_COMUNICACION', 
                      'LIMIT_AUDICION', 'LIMIT_APRENDIZAJE', 'LIMIT_RELACION']
    
    if all(col in df.columns for col in disability_cols):
        df['DISCAPACIDAD'] = df[disability_cols].sum(axis=1).apply(lambda x: 1 if x >= 1 else 0)
        df = df.drop(columns=disability_cols)
    
    # One-hot encoding para variables categóricas
    categorical_cols = ['ETNIA', 'TIPO_TRABAJADOR', 'NIVEL_OCUPACION', 
                       'REGISTRO_SUNAT', 'LIBROS_CONTABLES', 'FRECUENCIA_PAGO']
    
    for col in categorical_cols:
        if col in df.columns:
            df = pd.get_dummies(df, columns=[col], drop_first=True, dummy_na=True)
    
    # Columnas que el modelo espera (basado en el modelo cargado)
    expected_features = [
        'SEXO', 'EDAD', 'NIVEL_EDUCATIVO', 'TUVO_TRABAJO', 'BUSCA_OTRO_TRABAJO',
        'TAMANO_EMPRESA', 'QUIERE_MAS_HORAS', 'INGRESO_PRINCIPAL', 'SEGURO_SALUD',
        'DIAS_AUSENTE', 'DISCAPACIDAD', 'ETNIA_2', 'ETNIA_3', 'ETNIA_4', 'ETNIA_5',
        'ETNIA_6', 'ETNIA_7', 'ETNIA_8', 'ETNIA_9', 'TIPO_TRABAJADOR_2.0',
        'TIPO_TRABAJADOR_3.0', 'TIPO_TRABAJADOR_4.0', 'TIPO_TRABAJADOR_6.0',
        'TIPO_TRABAJADOR_7.0', 'TIPO_TRABAJADOR_8.0', 'TIPO_TRABAJADOR_9.0',
        'NIVEL_OCUPACION_2', 'NIVEL_OCUPACION_3', 'NIVEL_OCUPACION_4',
        'REGISTRO_SUNAT_2.0', 'REGISTRO_SUNAT_3.0', 'LIBROS_CONTABLES_2.0',
        'FRECUENCIA_PAGO_2.0', 'FRECUENCIA_PAGO_3.0', 'FRECUENCIA_PAGO_4.0',
        'FRECUENCIA_PAGO_5.0'
    ]
    
    # Asegurar que todas las columnas esperadas estén presentes
    for col in expected_features:
        if col not in df.columns:
            df[col] = 0
    
    # Retornar solo las columnas en el orden correcto
    return df[expected_features]

# 4. Aplicar preprocesamiento
print("🔧 Aplicando preprocesamiento...")
try:
    processed_data = preprocess_for_prediction(new_data)
    print(f"✅ Datos procesados: {processed_data.shape}")
    print(f"📋 Columnas procesadas: {list(processed_data.columns)}")
    
    # 5. Hacer predicción
    print("🎯 Realizando predicción...")
    clusters = pipeline.predict(processed_data)
    
    # 6. Agregar resultados a los datos originales
    new_data['Cluster'] = clusters
    
    # 7. Guardar resultados
    output_file = 'resultados_con_clusters.csv'
    new_data.to_csv(output_file, index=False)
    
    print("✅ ¡Predicción completada exitosamente!")
    print(f"📈 Distribución de clusters:")
    cluster_counts = pd.Series(clusters).value_counts().sort_index()
    for cluster, count in cluster_counts.items():
        print(f"   Cluster {cluster}: {count} registros")
    
    print(f"💾 Resultados guardados en: {output_file}")
    
    # Mostrar una muestra de los resultados
    print("\n📝 Muestra de resultados:")
    print(new_data[['SEXO', 'EDAD', 'NIVEL_EDUCATIVO', 'Cluster']].head())
    
except Exception as e:
    print(f"🚨 ERROR: {e}")
    print("\n🔍 Información de depuración:")
    print(f"Forma de los datos originales: {new_data.shape}")
    print(f"Columnas disponibles: {list(new_data.columns)}")

📦 Cargando modelo...


ModuleNotFoundError: No module named 'pipeline'

NUEVO MODELO

In [12]:
import sys
sys.path.append('..')  # PATH funcionando ✅

import pandas as pd
import joblib
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer

print("="*60)
print("🤖 USANDO EL MODELO NUEVO - kmeans_model.joblib")
print("="*60)

# 1. Cargar datos sin procesar
print("📊 Cargando datos sin procesar...")
try:
    datos_crudos = pd.read_csv('notebooks/new_data_sample_SINP.csv', na_values=[" "])
    
    print(f"✅ Datos crudos cargados: {datos_crudos.shape}")
    print(f"📋 Columnas disponibles: {list(datos_crudos.columns)}")
    
    # Mostrar vista previa
    print("\n📝 Vista previa de los datos:")
    print(datos_crudos.head())
    
except Exception as e:
    print(f"❌ Error cargando datos: {e}")
    datos_crudos = None

# 2. Crear función de preprocesamiento manual
def preprocess_raw_data(df):
    """
    Convierte datos crudos (códigos) a formato procesado
    """
    df = df.copy()
    
    # Mapeo de códigos a nombres descriptivos
    column_mapping = {
        "C207": "SEXO",
        "C208": "EDAD", 
        "C366": "NIVEL_EDUCATIVO",
        "C377": "ETNIA",
        "C303": "TUVO_TRABAJO",
        "C310": "TIPO_TRABAJADOR",
        "C335": "BUSCA_OTRO_TRABAJO",
        "OCUP300": "NIVEL_OCUPACION",
        "C312": "REGISTRO_SUNAT",
        "C317": "TAMANO_EMPRESA",
        "C333": "QUIERE_MAS_HORAS",
        "C313": "LIBROS_CONTABLES",
        "C338": "FRECUENCIA_PAGO",
        "ingtrabw": "INGRESO_PRINCIPAL",
        "C375_1": "LIMIT_MOVIMIENTO",
        "C375_2": "LIMIT_VISION",
        "C375_3": "LIMIT_COMUNICACION",
        "C375_4": "LIMIT_AUDICION",
        "C375_5": "LIMIT_APRENDIZAJE",
        "C375_6": "LIMIT_RELACION",
        "SEGURO1": "SEGURO_SALUD",
        "C205": "DIAS_AUSENTE"
    }
    
    # Renombrar columnas
    df = df.rename(columns=column_mapping)
    
    # Aplicar el mismo preprocesamiento que ya funciona
    # Recodificar variables binarias (2 -> 0)
    binary_cols = ['SEXO', 'TUVO_TRABAJO', 'BUSCA_OTRO_TRABAJO', 'QUIERE_MAS_HORAS', 'SEGURO_SALUD']
    for col in binary_cols:
        if col in df.columns:
            df[col] = df[col].replace({2: 0})
    
    # Crear variable DISCAPACIDAD
    disability_cols = ['LIMIT_MOVIMIENTO', 'LIMIT_VISION', 'LIMIT_COMUNICACION', 
                      'LIMIT_AUDICION', 'LIMIT_APRENDIZAJE', 'LIMIT_RELACION']
    
    if all(col in df.columns for col in disability_cols):
        df['DISCAPACIDAD'] = df[disability_cols].sum(axis=1).apply(lambda x: 1 if x >= 1 else 0)
        df = df.drop(columns=disability_cols)
    
    # One-hot encoding
    categorical_cols = ['ETNIA', 'TIPO_TRABAJADOR', 'NIVEL_OCUPACION', 
                       'REGISTRO_SUNAT', 'LIBROS_CONTABLES', 'FRECUENCIA_PAGO']
    
    for col in categorical_cols:
        if col in df.columns:
            df = pd.get_dummies(df, columns=[col], drop_first=True, dummy_na=True)
    
    # Columnas esperadas
    expected_features = [
        'SEXO', 'EDAD', 'NIVEL_EDUCATIVO', 'TUVO_TRABAJO', 'BUSCA_OTRO_TRABAJO',
        'TAMANO_EMPRESA', 'QUIERE_MAS_HORAS', 'INGRESO_PRINCIPAL', 'SEGURO_SALUD',
        'DIAS_AUSENTE', 'DISCAPACIDAD', 'ETNIA_2', 'ETNIA_3', 'ETNIA_4', 'ETNIA_5',
        'ETNIA_6', 'ETNIA_7', 'ETNIA_8', 'ETNIA_9', 'TIPO_TRABAJADOR_2.0',
        'TIPO_TRABAJADOR_3.0', 'TIPO_TRABAJADOR_4.0', 'TIPO_TRABAJADOR_6.0',
        'TIPO_TRABAJADOR_7.0', 'TIPO_TRABAJADOR_8.0', 'TIPO_TRABAJADOR_9.0',
        'NIVEL_OCUPACION_2', 'NIVEL_OCUPACION_3', 'NIVEL_OCUPACION_4',
        'REGISTRO_SUNAT_2.0', 'REGISTRO_SUNAT_3.0', 'LIBROS_CONTABLES_2.0',
        'FRECUENCIA_PAGO_2.0', 'FRECUENCIA_PAGO_3.0', 'FRECUENCIA_PAGO_4.0',
        'FRECUENCIA_PAGO_5.0'
    ]
    
    # Asegurar columnas
    for col in expected_features:
        if col not in df.columns:
            df[col] = 0
    
    return df[expected_features]

# 3. Probar el procesamiento
if datos_crudos is not None:
    print("\n🔧 Aplicando preprocesamiento manual...")
    try:
        datos_procesados = preprocess_raw_data(datos_crudos)
        print(f"✅ Datos procesados: {datos_procesados.shape}")
        
        # 4. Cargar el modelo NUEVO (kmeans_model.joblib)
        print("\n📦 Cargando modelo NUEVO...")
        modelo_nuevo = joblib.load('models/kmeans_model.joblib')
        print(f"✅ Modelo NUEVO cargado: {type(modelo_nuevo)}")
        
        # 5. Hacer predicción
        print("🎯 Realizando predicción con modelo NUEVO...")
        clusters = modelo_nuevo.predict(datos_procesados)
        
        # 6. Agregar resultados
        datos_crudos['Cluster'] = clusters
        
        # 7. Guardar resultados
        output_file = 'resultados_modelo_NUEVO.csv'
        datos_crudos.to_csv(output_file, index=False)
        
        print("✅ ¡Predicción exitosa con MODELO NUEVO!")
        print(f"📈 Distribución de clusters:")
        cluster_counts = pd.Series(clusters).value_counts().sort_index()
        for cluster, count in cluster_counts.items():
            print(f"   Cluster {cluster}: {count} registros")
        
        print(f"💾 Resultados guardados en: {output_file}")
        
        # Mostrar muestra de resultados
        print("\n📝 Muestra de resultados:")
        print("Primeras 5 filas:")
        display_cols = ['C207', 'C208', 'C366', 'Cluster']
        available_display = [col for col in display_cols if col in datos_crudos.columns]
        print(datos_crudos[available_display].head())
        
    except Exception as e:
        print(f"❌ Error: {e}")
        import traceback
        traceback.print_exc()

print("\n" + "="*60)

🤖 USANDO EL MODELO NUEVO - kmeans_model.joblib
📊 Cargando datos sin procesar...
✅ Datos crudos cargados: (25, 22)
📋 Columnas disponibles: ['C207', 'C208', 'C366', 'C377', 'C303', 'C310', 'C335', 'OCUP300', 'C312', 'C317', 'C333', 'C313', 'C338', 'ingtrabw', 'C375_1', 'C375_2', 'C375_3', 'C375_4', 'C375_5', 'C375_6', 'SEGURO1', 'C205']

📝 Vista previa de los datos:
   C207  C208  C366  C377  C303  C310  C335  OCUP300  C312  C317  ...  C338  \
0     2    24    10     7     2   NaN   NaN        4   NaN   NaN  ...   NaN   
1     1    78    11     1     2   NaN   NaN        4   NaN   NaN  ...   NaN   
2     1    21     8     7     1   3.0   2.0        1   2.0   3.0  ...   1.0   
3     2    22    10     7     2   NaN   NaN        4   NaN   NaN  ...   NaN   
4     1    43     6     7     1   3.0   2.0        1   1.0   2.0  ...   1.0   

   ingtrabw  C375_1  C375_2  C375_3  C375_4  C375_5  C375_6  SEGURO1  C205  
0       NaN       2       2       2       2       2       2        5   2.0  
1   

Traceback (most recent call last):
  File "C:\Users\destr\AppData\Local\Temp\ipykernel_14648\3589152920.py", line 127, in <module>
    clusters = modelo_nuevo.predict(datos_procesados)
  File "c:\Users\destr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\pipeline.py", line 787, in predict
    Xt = transform.transform(Xt)
  File "c:\Users\destr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\utils\_set_output.py", line 319, in wrapped
    data_to_wrap = f(self, X, *args, **kwargs)
  File "c:\Users\destr\AppData\Local\Programs\Python\Python313\Lib\site-packages\sklearn\compose\_column_transformer.py", line 1090, in transform
    raise ValueError(f"columns are missing: {diff}")
ValueError: columns are missing: {'TIPO_TRABAJADOR', 'ETNIA', 'LIBROS_CONTABLES', 'FRECUENCIA_PAGO', 'NIVEL_OCUPACION', 'REGISTRO_SUNAT'}


*ESCALES*

In [ ]:
# Ejemplo de uso
import joblib
import pandas as pd

# Cargar modelo
model = joblib.load('models/kmeans_model.joblib')

# Datos crudos (formato original)
nuevos_datos = pd.DataFrame({
    'SEXO': [1, 2, 1],  # 1=Hombre, 2=Mujer
    'EDAD': [25, 45, 35],  # Valores reales
    'INGRESO_PRINCIPAL': [1500, 2500, 1800],  # Valores reales
    'DIAS_AUSENTE': [0, 2, 1],
    'ETNIA': ['Mestizo', 'Indígena', 'Blanco'],
    'TIPO_TRABAJADOR': ['Empleado', 'Obrero', 'Independiente'],
    'NIVEL_OCUPACION': ['Profesional', 'Técnico', 'Operario'],
    'REGISTRO_SUNAT': [1, 2, 1],
    'LIBROS_CONTABLES': [1, 2, 1],
    'FRECUENCIA_PAGO': ['Mensual', 'Semanal', 'Quincenal'],
    'TUVO_TRABAJO': [1, 2, 1],
    'BUSCA_OTRO_TRABAJO': [1, 2, 1],
    'QUIERE_MAS_HORAS': [1, 2, 1],
    'SEGURO_SALUD': [1, 2, 1],
    'LIMIT_MOVIMIENTO': [1, 2, 1],
    'LIMIT_VISION': [1, 2, 1],
    'LIMIT_COMUNICACION': [1, 2, 1],
    'LIMIT_AUDICION': [1, 2, 1],
    'LIMIT_APRENDIZAJE': [1, 2, 1],
    'LIMIT_RELACION': [1, 2, 1]
})

# Predecir clusters
clusters = model.predict(nuevos_datos)
print(f"Clusters asignados: {clusters}")

📊 Datos originales:
(3, 22)
['SEXO', 'EDAD', 'NIVEL_EDUCATIVO', 'TAMANO_EMPRESA', 'INGRESO_PRINCIPAL', 'DIAS_AUSENTE', 'ETNIA', 'TIPO_TRABAJADOR', 'NIVEL_OCUPACION', 'REGISTRO_SUNAT', 'LIBROS_CONTABLES', 'FRECUENCIA_PAGO', 'TUVO_TRABAJO', 'BUSCA_OTRO_TRABAJO', 'QUIERE_MAS_HORAS', 'SEGURO_SALUD', 'LIMIT_MOVIMIENTO', 'LIMIT_VISION', 'LIMIT_COMUNICACION', 'LIMIT_AUDICION', 'LIMIT_APRENDIZAJE', 'LIMIT_RELACION']

🔧 Aplicando preprocesamiento...
✅ Datos procesados:
Forma: (3, 36)
Columnas: ['SEXO', 'EDAD', 'NIVEL_EDUCATIVO', 'TUVO_TRABAJO', 'BUSCA_OTRO_TRABAJO', 'TAMANO_EMPRESA', 'QUIERE_MAS_HORAS', 'INGRESO_PRINCIPAL', 'SEGURO_SALUD', 'DIAS_AUSENTE', 'DISCAPACIDAD', 'ETNIA_2', 'ETNIA_3', 'ETNIA_4', 'ETNIA_5', 'ETNIA_6', 'ETNIA_7', 'ETNIA_8', 'ETNIA_9', 'TIPO_TRABAJADOR_2.0', 'TIPO_TRABAJADOR_3.0', 'TIPO_TRABAJADOR_4.0', 'TIPO_TRABAJADOR_6.0', 'TIPO_TRABAJADOR_7.0', 'TIPO_TRABAJADOR_8.0', 'TIPO_TRABAJADOR_9.0', 'NIVEL_OCUPACION_2', 'NIVEL_OCUPACION_3', 'NIVEL_OCUPACION_4', 'REGISTRO_SUNAT_2

ValueError: columns are missing: {'TIPO_TRABAJADOR', 'ETNIA', 'LIBROS_CONTABLES', 'FRECUENCIA_PAGO', 'NIVEL_OCUPACION', 'REGISTRO_SUNAT'}

*EXTRAER DATA*

In [1]:
# extraer_muestra.ipynb
import pandas as pd
import numpy as np

# --- Configuración ---
RANDOM_STATE = 42  # Para reproducibilidad
SAMPLE_SIZE = 25   # Número de registros a extraer
ORIGINAL_CSV = '../data/raw/employ25_3-4-5.csv'  # Ruta a tu archivo original
OUTPUT_CSV = 'new_data_sample.csv'   # Archivo de salida

# --- Mapeo de nombres (el mismo que usaste en data_loader) ---
column_mapping = {
    "C207": "SEXO",
    "C208": "EDAD",
    "C366": "NIVEL_EDUCATIVO",
    "C377": "ETNIA",
    "C303": "TUVO_TRABAJO",
    "C310": "TIPO_TRABAJADOR",
    "C335": "BUSCA_OTRO_TRABAJO",
    "OCUP300": "NIVEL_OCUPACION",
    "C312": "REGISTRO_SUNAT",
    "C317": "TAMANO_EMPRESA",
    "C333": "QUIERE_MAS_HORAS",
    "C313": "LIBROS_CONTABLES",
    "C338": "FRECUENCIA_PAGO",
    "ingtrabw": "INGRESO_PRINCIPAL",
    "C375_1": "LIMIT_MOVIMIENTO",
    "C375_2": "LIMIT_VISION",
    "C375_3": "LIMIT_COMUNICACION",
    "C375_4": "LIMIT_AUDICION",
    "C375_5": "LIMIT_APRENDIZAJE",
    "C375_6": "LIMIT_RELACION",
    "SEGURO1": "SEGURO_SALUD",
    "C205": "DIAS_AUSENTE"
}

# --- 1. Cargar datos originales ---
print(f"📂 Cargando datos originales de {ORIGINAL_CSV}...")
df_original = pd.read_csv(ORIGINAL_CSV, na_values=[" "])

# --- 2. Limpieza básica ---
print("🧹 Realizando limpieza básica...")
# Eliminar columnas completamente vacías
df_original.dropna(axis=1, how='all', inplace=True)

# --- 3. Extraer muestra aleatoria ---
print(f"🎲 Extrayendo {SAMPLE_SIZE} registros aleatorios...")
sample = df_original.sample(SAMPLE_SIZE, random_state=RANDOM_STATE)

# --- 4. Seleccionar y renombrar columnas relevantes ---
print("🔍 Seleccionando y renombrando columnas relevantes...")

# Seleccionar solo las columnas que están en el mapeo
available_columns = [col for col in column_mapping.keys() if col in sample.columns]
sample = sample[available_columns]

# Renombrar columnas a nombres descriptivos
sample = sample.rename(columns=column_mapping)

# --- 5. Guardar muestra ---
print(f"💾 Guardando muestra en {OUTPUT_CSV}...")
sample.to_csv(OUTPUT_CSV, index=False)

# --- 6. Resultado final ---
print("\n✅ Muestra creada exitosamente con nombres descriptivos!")
print(f"• Total registros: {len(sample)}")
print(f"• Columnas: {sample.columns.tolist()}")
print("\n📝 Vista previa:")
display(sample.head())

📂 Cargando datos originales de ../data/raw/employ25_3-4-5.csv...
🧹 Realizando limpieza básica...
🎲 Extrayendo 25 registros aleatorios...
🔍 Seleccionando y renombrando columnas relevantes...
💾 Guardando muestra en new_data_sample.csv...

✅ Muestra creada exitosamente con nombres descriptivos!
• Total registros: 25
• Columnas: ['SEXO', 'EDAD', 'NIVEL_EDUCATIVO', 'ETNIA', 'TUVO_TRABAJO', 'TIPO_TRABAJADOR', 'BUSCA_OTRO_TRABAJO', 'NIVEL_OCUPACION', 'REGISTRO_SUNAT', 'TAMANO_EMPRESA', 'QUIERE_MAS_HORAS', 'LIBROS_CONTABLES', 'FRECUENCIA_PAGO', 'INGRESO_PRINCIPAL', 'LIMIT_MOVIMIENTO', 'LIMIT_VISION', 'LIMIT_COMUNICACION', 'LIMIT_AUDICION', 'LIMIT_APRENDIZAJE', 'LIMIT_RELACION', 'SEGURO_SALUD', 'DIAS_AUSENTE']

📝 Vista previa:


,SEXO,EDAD,NIVEL_EDUCATIVO,ETNIA,TUVO_TRABAJO,TIPO_TRABAJADOR,BUSCA_OTRO_TRABAJO,NIVEL_OCUPACION,REGISTRO_SUNAT,TAMANO_EMPRESA,...,FRECUENCIA_PAGO,INGRESO_PRINCIPAL,LIMIT_MOVIMIENTO,LIMIT_VISION,LIMIT_COMUNICACION,LIMIT_AUDICION,LIMIT_APRENDIZAJE,LIMIT_RELACION,SEGURO_SALUD,DIAS_AUSENTE
9757,2,24,10,7,2,NaN,NaN,4,NaN,NaN,...,NaN,NaN,2,2,2,2,2,2,5,2.0
9709,1,78,11,1,2,NaN,NaN,4,NaN,NaN,...,NaN,NaN,2,2,2,2,2,2,1,2.0
2588,1,21,8,7,1,3.0,2.0,1,2.0,3.0,...,1.0,1800.0,2,2,2,2,2,2,5,2.0
6451,2,22,10,7,2,NaN,NaN,4,NaN,NaN,...,NaN,NaN,2,2,2,2,2,2,5,2.0
518,1,43,6,7,1,3.0,2.0,1,1.0,2.0,...,1.0,4500.0,2,2,2,2,2,2,6,2.0


In [25]:
# extraer_muestra.ipynb
import pandas as pd
import numpy as np

# --- Configuración ---
RANDOM_STATE = 42  # Para reproducibilidad
SAMPLE_SIZE = 25   # Número de registros a extraer
ORIGINAL_CSV = '../data/raw/employ25_3-4-5.csv'  # Ruta a tu archivo original
OUTPUT_CSV = 'new_data_sample_SINP.csv'   # Archivo de salida

# --- 1. Cargar datos originales ---
print(f"📂 Cargando datos originales de {ORIGINAL_CSV}...")
df_original = pd.read_csv(ORIGINAL_CSV, na_values=[" "])

# --- 2. Limpieza básica ---
print("🧹 Realizando limpieza básica...")
# Eliminar columnas completamente vacías
df_original.dropna(axis=1, how='all', inplace=True)

# --- 3. Extraer muestra aleatoria ---
print(f"🎲 Extrayendo {SAMPLE_SIZE} registros aleatorios...")
sample = df_original.sample(SAMPLE_SIZE, random_state=RANDOM_STATE)

# --- 4. Seleccionar columnas relevantes ---
# Las mismas columnas que usaste en tu proyecto original
relevant_columns = [
    "C207", "C208", "C366", "C377", "C303", "C310", 
    "C335", "OCUP300", "C312", "C317", "C333", 
    "C313", "C338", "ingtrabw", "C375_1", "C375_2", 
    "C375_3", "C375_4", "C375_5", "C375_6", "SEGURO1", "C205"
]

print("🔍 Seleccionando columnas relevantes...")
sample = sample[relevant_columns]

# --- 5. Guardar muestra ---
print(f"💾 Guardando muestra en {OUTPUT_CSV}...")
sample.to_csv(OUTPUT_CSV, index=False)

# --- 6. Resultado final ---
print("\n✅ Muestra creada exitosamente!")
print(f"• Total registros: {len(sample)}")
print(f"• Columnas: {sample.columns.tolist()}")
print("\n📝 Vista previa:")
display(sample.head())

📂 Cargando datos originales de ../data/raw/employ25_3-4-5.csv...
🧹 Realizando limpieza básica...
🎲 Extrayendo 25 registros aleatorios...
🔍 Seleccionando columnas relevantes...
💾 Guardando muestra en new_data_sample_SINP.csv...

✅ Muestra creada exitosamente!
• Total registros: 25
• Columnas: ['C207', 'C208', 'C366', 'C377', 'C303', 'C310', 'C335', 'OCUP300', 'C312', 'C317', 'C333', 'C313', 'C338', 'ingtrabw', 'C375_1', 'C375_2', 'C375_3', 'C375_4', 'C375_5', 'C375_6', 'SEGURO1', 'C205']

📝 Vista previa:


,C207,C208,C366,C377,C303,C310,C335,OCUP300,C312,C317,...,C338,ingtrabw,C375_1,C375_2,C375_3,C375_4,C375_5,C375_6,SEGURO1,C205
9757,2,24,10,7,2,NaN,NaN,4,NaN,NaN,...,NaN,NaN,2,2,2,2,2,2,5,2.0
9709,1,78,11,1,2,NaN,NaN,4,NaN,NaN,...,NaN,NaN,2,2,2,2,2,2,1,2.0
2588,1,21,8,7,1,3.0,2.0,1,2.0,3.0,...,1.0,1800.0,2,2,2,2,2,2,5,2.0
6451,2,22,10,7,2,NaN,NaN,4,NaN,NaN,...,NaN,NaN,2,2,2,2,2,2,5,2.0
518,1,43,6,7,1,3.0,2.0,1,1.0,2.0,...,1.0,4500.0,2,2,2,2,2,2,6,2.0
